In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from math import ceil
from tqdm import tqdm_notebook
import tqdm
from collections import Counter
from operator import itemgetter
from datetime import date, datetime

import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import json

## First Chanell

In [ ]:
months = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}
times = [['000000', '080000'], ['080000', '160000'],['160000', '235959']]
tv1_20 = dict()
for month in tqdm_notebook(list(months.keys())[:2]):
    for day in tqdm_notebook(list(range(months[month]))):
        date = f'2020{month:02d}{day+1:02d}'
        tv1_20[f'2020.{month:02d}.{day+1:02d}'] = []
        for time in times:
            try:
                url = f'https://api.gdeltproject.org/api/v2/doc/doc?query=%20(domain:.1tv.ru%20OR%20domainis:1tv.ru)%20sourcelang:rus&mode=ArtList&maxrecords=250&sort=DateDesc&format=html&startdatetime={date}{time[0]}&enddatetime={date}{time[1]}'
                r = requests.get(url)
                soup = BeautifulSoup(r.text)
                tv1_20[f'2020.{month:02d}.{day+1:02d}'].extend([a['href'] for a in soup.body.table.find_all('a')])
                print(f'2020.{month:02d}.{day+1:02d}', len(soup.body.table.find_all('a')))
            except (AttributeError, IndexError):
                print(date)
with open('tv1_links_2020.json', 'w', encoding = 'utf-8') as f:
    json.dump(tv1_20, f)

In [ ]:
tv1_2020 = []
for date in tqdm_notebook(list(tv1_20.keys())):
    for link in tqdm_notebook(tv1_20[date]):
#         if link not in pd.DataFrame(tv1_2020).Link:
        try:
            soup = BeautifulSoup(requests.get(link).text)
            tv1_2020.append({'Media':'1tv.ru', 'Date':date, 'Link':link, 
                             'Text':' '.join([soup.h1.text,
                                              ' '.join([p.text for p in soup
                                                        .find('div', 'editor text-block active')
                                                        .find_all('p')])]).replace('\xa0', ' '), 
                             'Dep':'', 'Tags':[]})
        except (requests.exceptions.ConnectionError, AttributeError, 
            IndexError, requests.exceptions.TooManyRedirects, ValueError, TypeError):
            print(link)
    if date.endswith('1'):
        pd.DataFrame(tv1_2020).to_csv('tv1_2020.csv', encoding='utf-8')
pd.DataFrame(tv1_2020).to_csv('tv1_2020.csv', encoding='utf-8')
pd.DataFrame(tv1_2020)

## NTV

In [ ]:
months = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}
ntv_20 = dict()
for month in tqdm_notebook(list(months.keys())[:2]):
    for day in tqdm_notebook(list(range(months[month]))):
        date = f'2020{month:02d}{day+1:02d}'
        ntv_20[f'2020.{month:02d}.{day+1:02d}'] = []
        try:
            url = f'https://www.ntv.ru/rest/news/day.jsp?dt={date}'
            r = requests.get(url)
            soup = BeautifulSoup(r.text)
            ntv_20[f'2020.{month:02d}.{day+1:02d}'] = ['https://www.ntv.ru'+a['href'] for a in soup.find('div', 'day_news').find_all('a') if 'novosti' in a['href']]
            print(f'2020.{month:02d}.{day+1:02d}', len(ntv_20[f'2020.{month:02d}.{day+1:02d}']))
        except (AttributeError, IndexError):
            print(date)
with open('ntv_links_2020.json', 'w', encoding = 'utf-8') as f:
    json.dump(ntv_20, f)

In [ ]:
ntv_2020 = []
for date in tqdm_notebook(list(ntv_20.keys())):
    for link in tqdm_notebook(ntv_20[date]):
#         if link not in pd.DataFrame(echo_2020).Link:
        try:
            r = requests.get(link)
            soup = BS(r.text)
            ntv_2020.append({'Media':'ntv.ru', 'Date':date, 'Link':link,
                                  'Text':' '.join([soup.h1.text, ' '.join([p.text for p in soup.find('div', 'inpagebody').find_all('p')])]).replace('\xa0', ' '), 
                                  'Dep':'', 'Tags':[]})
        except (requests.exceptions.ConnectionError, AttributeError, 
                IndexError, requests.exceptions.TooManyRedirects, ValueError):
            print(link)
    if date.endswith('1'):
        pd.DataFrame(ntv_2020).to_csv('ntv_2020.csv', encoding='utf-8')
pd.DataFrame(ntv_2020).to_csv('ntv_2020.csv', encoding='utf-8')
pd.DataFrame(ntv_2020)

## Zvezda

In [ ]:
months = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}
times = [['000000', '120000'], ['120000', '235959']]
zvezda_20 = dict()
for month in tqdm_notebook(list(months.keys())[:2]):
    for day in tqdm_notebook(list(range(months[month]))):
        date = f'2020{month:02d}{day+1:02d}'
        zvezda_20[f'2020.{month:02d}.{day+1:02d}'] = []
        for time in times:
            try:
                url = f'https://api.gdeltproject.org/api/v2/doc/doc?query=%20(domain:.tvzvezda.ru%20OR%20domainis:tvzvezda.ru)%20sourcelang:rus&mode=ArtList&maxrecords=250&sort=DateDesc&format=html&startdatetime={date}{time[0]}&enddatetime={date}{time[1]}'
#                 url = f'https://api.gdeltproject.org/api/v2/doc/doc?query=%20(domain:.mk.ru%20OR%20domainis:mk.ru)%20sourcelang:rus&mode=ArtList&maxrecords=250&sort=DateDesc&format=html&startdatetime={date}{time[0]}&enddatetime={date}{time[1]}'
    #             url = f'https://api.gdeltproject.org/api/v2/doc/doc?query=%20(domain:.kp.ru%20OR%20domainis:kp.ru)%20sourcelang:rus&mode=ArtList&maxrecords=250&sort=DateDesc&format=html&startdatetime={date}{time[0]}&enddatetime={date}{time[1]}'
                r = requests.get(url)
                soup = BeautifulSoup(r.text)
                zvezda_20[f'2020.{month:02d}.{day+1:02d}'].extend([a['href'] for a in soup.body.table.find_all('a')])
                print(f'2020.{month:02d}.{day+1:02d}', len(soup.body.table.find_all('a')))
            except (AttributeError, IndexError):
                print(date)
with open('zvezda_links_2020.json', 'w', encoding = 'utf-8') as f:
    json.dump(zvezda_20, f)

In [ ]:
tvzvezda_2020 = []
for date in tqdm_notebook(list(zvezda_20.keys())):
    for link in tqdm_notebook(zvezda_20[date]):
        try:
            r = requests.get(link)
            soup = BeautifulSoup(r.text)
            tvzvezda_2020.append({'Media':'tvzvezda.ru', 'Date':date, 'Link':link,
                                  'Text':' '.join([soup.h1.text, ' '.join([p.text for p in soup.find('div', 'glav_text').find_all('p')])]), 
                                  'Dep':'', 'Tags':[]})
        except (requests.exceptions.ConnectionError, AttributeError, 
                IndexError, requests.exceptions.TooManyRedirects, ValueError):
            print(link)
    if date.endswith('1'):
        pd.DataFrame(tvzvezda_2020).to_csv('tvzvezda_2020.csv', encoding='utf-8')
pd.DataFrame(tvzvezda_2020).to_csv('tvzvezda_2020.csv', encoding='utf-8')
pd.DataFrame(tvzvezda_2020)

## Dozhd'

In [ ]:
months = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}
tvrain_20 = dict()
year = '2020'
for month in tqdm_notebook(list(months.keys())[:2]):
    for day in tqdm_notebook(list(range(months[month]))):
#         date = f'2020{month:02d}{day+1:02d}'
        links_temp = list()
        url = f'https://tvrain.ru/archive/?search_teleshow_cat=&search_year={year}&search_month={month}&search_day={day+1}&query=&page={page+1}'
        r = requests.get(url)
        soup = BeautifulSoup(r.text)
        num_of_pages = ceil(int(soup.find('a', 'filter__items__item--label_brand').text.strip().split()[-1])/20)
        print('num_of_news', int(soup.find('a', 'filter__items__item--label_brand').text.strip().split()[-1]),
             'num_of pages', num_of_pages)
        for page in range(num_of_pages):
            try:
                url = f'https://tvrain.ru/archive/?search_teleshow_cat=&search_year={year}&search_month={month}&search_day={day}&query=&page={page+1}'
                r = requests.get(url)
                soup = BeautifulSoup(r.text)
                links_temp.extend(['https://tvrain.ru'+a['href'] for a in soup.find_all('a', 'chrono_list__item__info__name')])
                print(f'2020.{month:02d}.{day+1:02d}', len(links_temp))
#                 page+=1
            except (AttributeError, IndexError):
                print(date)
#             if len(links_temp) == number:
#                 break
        tvrain_20[f'2020.{month:02d}.{day+1:02d}'] = links_temp
        
with open('tvrain_links_2020.json', 'w', encoding = 'utf-8') as f:
    json.dump(tvrain_20, f)

In [ ]:
tvrain_2020 = []
for date in tqdm_notebook(list(tvrain_20.keys())):
    for link in tqdm_notebook(tvrain_20[date]):
        try:
            r = requests.get(link)
            soup = BS(r.text)
            tvrain_2020.append({'Media':'tvrain.ru', 'Date':date, 'Link':link,
                                  'Text':' '.join([soup.h1.text, ' '.join([p.text for p in soup.find('div', id='article_content_text').find_all('p') if 'донейт' not in p.text])]).replace('\xa0', ' ').replace('\n\n', ' '), 
                                  'Dep':'', 'Tags':[]})
        except (requests.exceptions.ConnectionError, AttributeError, 
                IndexError, requests.exceptions.TooManyRedirects, ValueError):
            print(link)
    if date.endswith('1'):
        pd.DataFrame(tvrain_2020).to_csv('tvrain_2020.csv', encoding='utf-8')
pd.DataFrame(tvrain_2020).to_csv('tvrain_2020.csv', encoding='utf-8')
pd.DataFrame(tvrain_2020)

## Govorit Moskva

In [ ]:
months = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}
times = [['000000', '120000'], ['120000', '235959']]
govoritmoskva_20 = dict()
for month in tqdm_notebook(list(months.keys())[:2]):
    for day in tqdm_notebook(list(range(months[month]))):
        date = f'2020{month:02d}{day+1:02d}'
        govoritmoskva_20[f'2020.{month:02d}.{day+1:02d}'] = []
        for time in times:
            try:
                url = f'https://api.gdeltproject.org/api/v2/doc/doc?query=%20(domain:.govoritmoskva.ru%20OR%20domainis:govoritmoskva.ru)%20sourcelang:rus&mode=ArtList&maxrecords=250&sort=DateDesc&format=html&startdatetime={date}{time[0]}&enddatetime={date}{time[1]}'
                r = requests.get(url)
                soup = BeautifulSoup(r.text)
                govoritmoskva_20[f'2020.{month:02d}.{day+1:02d}'].extend([a['href'] for a in soup.body.table.find_all('a')])
                print(f'2020.{month:02d}.{day+1:02d}', len(soup.body.table.find_all('a')))
            except (AttributeError, IndexError):
                print(date)
with open('govoritmoskva_links_2020.json', 'w', encoding = 'utf-8') as f:
    json.dump(govoritmoskva_20, f)

In [ ]:
govoritmoskva_2020 = []
for date in tqdm_notebook(list(govoritmoskva_20.keys())):
    for link in tqdm_notebook(govoritmoskva_20[date]):
        try:
            r = requests.get(link)
            soup = BeautifulSoup(r.text)
            govoritmoskva_2020.append({'Media':'govoritmoskva.ru', 'Date':date, 'Link':link,
                                  'Text':' '.join([soup.h1.text, ' '.join([p.text for p in soup.find('div', 'textContent').find_all('p')]).strip()]), 
                                  'Dep':'', 'Tags':[]})
        except (requests.exceptions.ConnectionError, AttributeError, 
                IndexError, requests.exceptions.TooManyRedirects, ValueError):
            print(link)
    if date.endswith('1'):
        pd.DataFrame(govoritmoskva_2020).to_csv('govoritmoskva_2020.csv', encoding='utf-8')
pd.DataFrame(govoritmoskva_2020).to_csv('govoritmoskva_2020.csv', encoding='utf-8')
pd.DataFrame(govoritmoskva_2020)

## Echo

In [ ]:
months = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}
times = [['000000', '120000'], ['120000', '235959']]
echo_20 = dict()
for month in tqdm_notebook(list(months.keys())[:2]):
    for day in tqdm_notebook(list(range(months[month]))):
        date = f'2020{month:02d}{day+1:02d}'
        echo_20[f'2020.{month:02d}.{day+1:02d}'] = []
        for time in times:
            try:
                url = f'https://api.gdeltproject.org/api/v2/doc/doc?query=%20(domain:.echo.msk.ru%20OR%20domainis:echo.msk.ru)%20sourcelang:rus&mode=ArtList&maxrecords=250&sort=DateDesc&format=html&startdatetime={date}{time[0]}&enddatetime={date}{time[1]}'
                r = requests.get(url)
                soup = BeautifulSoup(r.text)
                echo_20[f'2020.{month:02d}.{day+1:02d}'].extend([a['href'] for a in soup.body.table.find_all('a')])
                print(f'2020.{month:02d}.{day+1:02d}', len(soup.body.table.find_all('a')))
            except (AttributeError, IndexError):
                print(date)
with open('echo_links_2020.json', 'w', encoding = 'utf-8') as f:
    json.dump(echo_20, f)

In [ ]:
echo_2020 = []
for date in tqdm_notebook(list(echo_20.keys())):
    for link in tqdm_notebook(echo_20[date]):
        if 'comments' not in link and 'programs' not in link:
            try:
                r = requests.get(link)
                soup = BeautifulSoup(r.text)
                echo_2020.append({'Media':'echo.msk.ru', 'Date':date, 'Link':link,
                                      'Text':' '.join([soup.h1.text, ' '.join([p.text.strip() for p in soup.find('div', 'typical').find_all('p')])]), 
                                      'Dep':'', 'Tags':[]})
            except (requests.exceptions.ConnectionError, AttributeError, 
                    IndexError, requests.exceptions.TooManyRedirects, ValueError):
                print(link)
    if date.endswith('1'):
        pd.DataFrame(echo_2020).to_csv('echo_2020.csv', encoding='utf-8')
pd.DataFrame(echo_2020).to_csv('echo_2020.csv', encoding='utf-8')
pd.DataFrame(echo_2020)

## Svoboda

In [ ]:
months = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}
times = [['000000', '120000'], ['120000', '235959']]
svoboda_20 = dict()
for month in tqdm_notebook(list(months.keys())[:2]):
    for day in tqdm_notebook(list(range(months[month]))):
        date = f'2020{month:02d}{day+1:02d}'
        svoboda_20[f'2020.{month:02d}.{day+1:02d}'] = []
        for time in times:
            try:
                url = f'https://api.gdeltproject.org/api/v2/doc/doc?query=%20(domain:.svoboda.org%20OR%20domainis:svoboda.org)%20sourcelang:rus&mode=ArtList&maxrecords=250&sort=DateDesc&format=html&startdatetime={date}{time[0]}&enddatetime={date}{time[1]}'
                r = requests.get(url)
                soup = BeautifulSoup(r.text)
                svoboda_20[f'2020.{month:02d}.{day+1:02d}'].extend([a['href'] for a in soup.body.table.find_all('a')])
                print(f'2020.{month:02d}.{day+1:02d}', len(soup.body.table.find_all('a')))
            except (AttributeError, IndexError):
                print(date)
with open('svoboda_links_2020.json', 'w', encoding = 'utf-8') as f:
    json.dump(svoboda_20, f)

In [ ]:
svoboda_2020 = []
for date in tqdm_notebook(list(svoboda_20.keys())):
    for link in tqdm_notebook(svoboda_20[date]):
#         if link not in pd.DataFrame(echo_2020).Link:
        try:
            r = requests.get(link)
            soup = BeautifulSoup(r.text)
            svoboda_2020.append({'Media':'svoboda.org', 'Date':date, 'Link':link,
                                  'Text':' '.join([soup.h1.text.strip(), ' '.join([p.text for p in soup.find('div', 'wsw').find_all('p')])]), 
                                  'Dep':'', 'Tags':[]})
        except (requests.exceptions.ConnectionError, AttributeError, 
                IndexError, requests.exceptions.TooManyRedirects, ValueError):
            print(link)
    if date.endswith('1'):
        pd.DataFrame(svoboda_2020).to_csv('svoboda_2020.csv', encoding='utf-8')
pd.DataFrame(svoboda_2020).to_csv('svoboda_2020.csv', encoding='utf-8')
pd.DataFrame(svoboda_2020)

## Vesti FM

In [ ]:
months = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}
times = [['000000', '120000'], ['120000', '235959']]
radiovesti_20 = dict()
for month in tqdm_notebook(list(months.keys())[:2]):
    for day in tqdm_notebook(list(range(months[month]))):
        date = f'2020{month:02d}{day+1:02d}'
        radiovesti_20[f'2020.{month:02d}.{day+1:02d}'] = []
        for time in times:
            try:
                url = f'https://api.gdeltproject.org/api/v2/doc/doc?query=%20(domain:.radiovesti.ru%20OR%20domainis:radiovesti.ru)%20sourcelang:rus&mode=ArtList&maxrecords=250&sort=DateDesc&format=html&startdatetime={date}{time[0]}&enddatetime={date}{time[1]}'
                r = requests.get(url)
                soup = BeautifulSoup(r.text)
                radiovesti_20[f'2020.{month:02d}.{day+1:02d}'].extend([a['href'] for a in soup.body.table.find_all('a')])
                print(f'2020.{month:02d}.{day+1:02d}', len(soup.body.table.find_all('a')))
            except (AttributeError, IndexError):
                print(date)
with open('radiovesti_links_2020.json', 'w', encoding = 'utf-8') as f:
    json.dump(radiovesti_20, f)

In [ ]:
radiovesti_2020 = []
for date in tqdm_notebook(list(radiovesti_20.keys())):
    for link in tqdm_notebook(radiovesti_20[date]):
        try:
            r = requests.get(link)
            if r.status_code==200:
                r.encoding = 'utf-8'
                soup = BS(r.text)
                radiovesti_2020.append({'Media':'radiovesti.ru', 'Date':date, 'Link':link,
                                      'Text':' '.join([soup.h1.text, ' '.join([p.text for p in soup.find('div', 'insides-page__news__text').find_all('p')])]), 
                                      'Dep':'', 'Tags':[]})
        except (requests.exceptions.ConnectionError, AttributeError, 
                IndexError, requests.exceptions.TooManyRedirects, ValueError):
            print(link)
    if date.endswith('1'):
        pd.DataFrame(radiovesti_2020).to_csv('radiovesti_2020.csv', encoding='utf-8')
pd.DataFrame(radiovesti_2020).to_csv('radiovesti_2020.csv', encoding='utf-8')
pd.DataFrame(radiovesti_2020)